## for esa world cover 10m
- 10  Tree cover
- 20  Shrubland
- 30  Grassland
- 40  Cropland
- 50  Built-up
- 60  Bare/sparse vegetation
- 70  Snow and ice
- 80  Permanent water bodies
- 90  Herbaceous wetland
- 95  Mangroves
- 100 Moss and lichen

In [1]:
from eumap.misc import find_files, nan_percentile
from eumap.raster import read_rasters, save_rasters, write_new_raster
import time as tm
import numpy as np
import os
# import ee
# import geemap
import multiprocessing
from eumap.raster import read_rasters
import rasterio
from rasterio import features
import geopandas as gpd
from osgeo import ogr, gdal
from multiprocessing import Pool
import warnings
from os.path import exists

# Suppress all warnings
warnings.filterwarnings("ignore")
# Restore warning behavior (optional)
# warnings.filterwarnings("default")

### read in necessary shapefiles

In [2]:
# shape file from: EuroWorldMap, https://www.mapsforeurope.org/datasets/euro-global-map
# vec = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/LandmaskA.shp')
vec1 = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/LakeresA.shp')
vec2 = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/rivers3.shp')
# vec3 = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/SeaA.shp')
geo1 = vec1['geometry'].tolist()
geo2 = vec2['geometry'].tolist()
# sea = vec3['geometry'].tolist()
inwater = geo1+geo2
# inland = vec['geometry'].tolist()  # Convert geometries to a list

# exclude countries out of mapping scope
land = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/LandmaskA.shp')
ex = ['RU', 'GL', 'AZ', 'BY', 'GE', 'PM', 'AM', 'SX', 'BL', 'MF']
for i in ex:
    if i in land['ICC'].unique():
        land = land.drop(land.loc[land['ICC']==i].index)

gland = land['geometry'].tolist()

sea = gpd.read_file('/mnt/inca/EU_landmask/EuroGlobalMap_shapefile/SeaA.shp')
gsea = sea['geometry'].tolist()

### create a landmask base, with land (1), ocean (3) and inland water (2)

In [7]:
def worker_mask(inwater,gsea,gland,f):
    start = tm.time()
    mask_ds = rasterio.open(f)
    land_mask, _ = read_rasters(raster_files=[f], dtype='float32',n_jobs=40)
    out_f = '/mnt/inca/EU_landmask/landmask3/landmask_EU' + f[55::]
    if exists(out_f):
        return None
    # Create empty mask
    mask = np.zeros(mask_ds.shape)
    land_mask = np.squeeze(land_mask)
    land_mask[land_mask==80]=2
    land_mask[(land_mask!=2) & ~np.isnan(land_mask)]=1
    # assign land = 1
    island = features.geometry_mask(gland, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
    island = ~island
    mask[island] = 1
    # assign sea = 3
    issea = features.geometry_mask(gsea, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
    issea = ~issea
    mask[issea] = 3
    # assign inland water = 2
    isinwater = features.geometry_mask(inwater, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
    isinwater = ~isinwater
    mask[(land_mask==2) & (isinwater) & (mask==1)] = 2
    # assign possible land, according to world cover
    mask[(land_mask==1) & (mask==3)] = 1
    
#     if ~np.any(is_inland):
#         a = f.split('_')[-2]
#         print(f'{a} out of scope')
#         return a
#     out_t = '/mnt/inca/EU_landmask/landmask/test_inland' + f[55::]
#     save_rasters(f, [out_t], is_inland, n_jobs=30)
    
    save_rasters(f, [out_f], mask, n_jobs=30)
    end = tm.time()
    print(f'{round((end-start)/60,2)} mins used for {f[70:-8]}')
    return None

In [10]:
# base raster: World Cover, https://esa-worldcover.org/en
raster_files = find_files('/mnt/inca/EU_landmask/landmask2','*.tif')
coor = [str(i).split('_')[-2] for i in raster_files]
files = [f'/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_{i}_Map.tif' for i in coor]
inputs = [(inwater,gsea,gland,i) for i in files]

# test = ['/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N39E042_Map.tif']
# #        '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N33E033_Map.tif',
# #         '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N36E036_Map.tif',
# #         '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N33E036_Map.tif']
# for i in test:
#     worker_mask(inwater,gsea,gland,i)

with Pool(20) as p:
    result = p.starmap(worker_mask, inputs)

### Mend tiles with incomplete land mask

In [15]:
def worker_mend(inwater,gsea,gland,f):
    start = tm.time()
    mask_ds = rasterio.open(f)
    land_mask, _ = read_rasters(raster_files=[f], dtype='float32',n_jobs=40)
    out_f = '/mnt/inca/EU_landmask/landmask3/landmask_EU' + f[55::]
#     if exists(out_f):
#         return None
    # Create empty mask
    mask = np.zeros(mask_ds.shape)
    land_mask = np.squeeze(land_mask)
    land_mask[land_mask==80]=2
    land_mask[(land_mask!=2) & ~np.isnan(land_mask)]=1
    # assign land = 1
#     island = features.geometry_mask(gland, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
#     island = ~island
    mask[land_mask==1] = 1
    # assign sea = 3
    issea = features.geometry_mask(gsea, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
    issea = ~issea
    mask[issea] = 3
    # assign inland water = 2
#     isinwater = features.geometry_mask(inwater, out_shape=np.squeeze(mask).shape, transform=mask_ds.transform)
#     isinwater = ~isinwater
    mask[(land_mask==2)] = 2
    # assign possible land, according to world cover
    mask[(land_mask==1) & (mask==3)] = 1
    
#     if ~np.any(is_inland):
#         a = f.split('_')[-2]
#         print(f'{a} out of scope')
#         return a
#     out_t = '/mnt/inca/EU_landmask/landmask/test_inland' + f[55::]
#     save_rasters(f, [out_t], is_inland, n_jobs=30)
    
    save_rasters(f, [out_f], mask, n_jobs=30)
    end = tm.time()
    print(f'{round((end-start)/60,2)} mins used for {f[70:-8]}')
    return None

In [16]:
test = ['/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N45E018_Map.tif']
#        '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N33E033_Map.tif',
#         '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N36E036_Map.tif',
#         '/mnt/inca/EU_landmask/esa_worldcover_10m/ESA_WorldCover_10m_2021_v200_N33E036_Map.tif']
for i in test:
    worker_mend(inwater,gsea,gland,i)

1.63 mins used for N45E018


### build vrt to do visual check

In [18]:
input_files = find_files('/mnt/inca/EU_landmask/landmask3','*.tif')
input_files = [str(i) for i in input_files]
output_vrt = '/mnt/inca/EU_landmask/landmask3/eu_landmask.vrt'
ds = gdal.BuildVRT(output_vrt, input_files)
ds.FlushCache()

In [1]:
vrt_file = '/mnt/inca/EU_landmask/landmask2/eu_landmask.vrt'
tif_file = '/mnt/inca/Xuemeng_testing/eu_landmask2.tif'
ref_file = '/mnt/inca/EU_landmask/U2018_CHA1218_12_V2020_20u1.tif'
with rasterio.open(ref_file) as src:
    projection = src.crs
    transform = src.bounds

gdal.SetConfigOption('CHECK_DISK_FREE_SPACE', 'FALSE')
gdal.Warp(tif_file, vrt_file, format='GTiff', outputBounds=transform, dstSRS=projection, xRes=10, yRes=-10)
